<a href="https://colab.research.google.com/github/anadiedrichs/TP-LLMs/blob/main/TP_LLMs_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP



## Libraries



### Installation

In [44]:
!pip install langchain
!pip install langchain_community
!pip install langchain-core
!pip install PyPDF
!pip install sentence-transformers
!pip install chromadb

### Load the libraries

In [2]:
import argparse
import os
import shutil
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.document import Document
#from get_embedding_function import get_embedding_function
from langchain.vectorstores.chroma import Chroma

## Environment variables

In [3]:
CHROMA_PATH = "chroma"
DATA_PATH = "data"

# CHROMA_DATA_PATH = "chroma_data/"
EMBED_MODEL = "all-MiniLM-L6-v2"
COLLECTION_NAME = "biblioteca"

## Dataset

In [4]:
links = ["https://editorial.mingeneros.gob.ar:8080/xmlui/bitstream/handle/123456789/32/Violencias%20por%20motivos%20de%20g%c3%a9nero%20-%20MMGyD.pdf",
         "https://editorial.mingeneros.gob.ar:8080/xmlui/bitstream/handle/123456789/26/Perspectiva%20de%20g%c3%a9nero%20y%20diversidad%20-%20MMGyD.pdf",
         "https://editorial.mingeneros.gob.ar:8080/xmlui/bitstream/handle/123456789/19/Masculinidades%20sin%20violencia%20-%20MMGyD.pdf",
         "https://editorial.mingeneros.gob.ar:8080/xmlui/bitstream/handle/123456789/18/Igualdad%20en%20los%20cuidados.pdf",
         "https://editorial.mingeneros.gob.ar:8080/xmlui/bitstream/handle/123456789/35/Diversidad%20-%20MMGyD.pdf"
         ]
titles = ["Violencia por motivos de género",
        "Perspectiva de género y diversidad",
        "Masculinidades sin violencia",
        "Igualdad en los cuidados",
        "Diversidad una perspectiva para la igualdad"]
file_name = ["violencia.pdf",
        "perspectiva.pdf",
        "masculinidades.pdf",
        "cuidados.pdf",
        "diversidad.pdf"]
pag_from = [14-1,14-1,14-1,14-1,12-1] # resto uno porque se indexa desde cero
pag_to = [85,97,30,98,87]


In [5]:
# create a pandas Dataframe using the previous information

import pandas as pd

data = {
    "title": titles,
    "link": links,
    "file_name": file_name,
    "pag_from": pag_from,
    "pag_to": pag_to,
}

df = pd.DataFrame(data)

print(df)


                                         title  \
0              Violencia por motivos de género   
1           Perspectiva de género y diversidad   
2                 Masculinidades sin violencia   
3                     Igualdad en los cuidados   
4  Diversidad una perspectiva para la igualdad   

                                                link           file_name  \
0  https://editorial.mingeneros.gob.ar:8080/xmlui...       violencia.pdf   
1  https://editorial.mingeneros.gob.ar:8080/xmlui...     perspectiva.pdf   
2  https://editorial.mingeneros.gob.ar:8080/xmlui...  masculinidades.pdf   
3  https://editorial.mingeneros.gob.ar:8080/xmlui...        cuidados.pdf   
4  https://editorial.mingeneros.gob.ar:8080/xmlui...      diversidad.pdf   

   pag_from  pag_to  
0        13      85  
1        13      97  
2        13      30  
3        13      98  
4        11      87  


Descarga de los archivos PDF a directorio local DATA_PATH

In [6]:
# download pdfs using info from df variable and save then in data_path
import os

# Create the directory if it doesn't exist
os.makedirs(DATA_PATH, exist_ok=True)

# Download each PDF file
for index, row in df.iterrows():
    file_path = os.path.join(DATA_PATH, row["file_name"])
    !wget -O {file_path} {row["link"]}


--2024-05-22 21:55:40--  https://editorial.mingeneros.gob.ar:8080/xmlui/bitstream/handle/123456789/32/Violencias%20por%20motivos%20de%20g%c3%a9nero%20-%20MMGyD.pdf
Resolving editorial.mingeneros.gob.ar (editorial.mingeneros.gob.ar)... 181.209.31.64
Connecting to editorial.mingeneros.gob.ar (editorial.mingeneros.gob.ar)|181.209.31.64|:8080... connected.
HTTP request sent, awaiting response... 200 
Length: 1087323 (1.0M) [application/pdf]
Saving to: ‘data/violencia.pdf’

data/violencia.pdf  100%[===================>]   1.04M  1.21MB/s    in 0.9s    

2024-05-22 21:55:42 (1.21 MB/s) - ‘data/violencia.pdf’ saved [1087323/1087323]

--2024-05-22 21:55:42--  https://editorial.mingeneros.gob.ar:8080/xmlui/bitstream/handle/123456789/26/Perspectiva%20de%20g%c3%a9nero%20y%20diversidad%20-%20MMGyD.pdf
Resolving editorial.mingeneros.gob.ar (editorial.mingeneros.gob.ar)... 181.209.31.64
Connecting to editorial.mingeneros.gob.ar (editorial.mingeneros.gob.ar)|181.209.31.64|:8080... connected.
HTTP req

Check the downloaded pdf files

In [7]:
!ls -la ./data/*

-rw-r--r-- 1 root root   579487 Oct 20  2023 ./data/cuidados.pdf
-rw-r--r-- 1 root root   579487 Oct 20  2023 ./data/cuidados.txt
-rw-r--r-- 1 root root 13206566 Jul  3  2023 ./data/diversidad.pdf
-rw-r--r-- 1 root root 13206566 Jul  3  2023 ./data/diversidad.txt
-rw-r--r-- 1 root root   467060 Oct 20  2023 ./data/masculinidades.pdf
-rw-r--r-- 1 root root   467060 Oct 20  2023 ./data/masculinidades.txt
-rw-r--r-- 1 root root  3231581 Aug 14  2023 ./data/perspectiva.pdf
-rw-r--r-- 1 root root  3231581 Aug 14  2023 ./data/perspectiva.txt
-rw-r--r-- 1 root root  1087323 Jul 10  2023 ./data/violencia.pdf
-rw-r--r-- 1 root root  1087323 Jul 10  2023 ./data/violencia.txt


In [8]:
# Load the pdf files from DATA_PATH directory using df information. We have to load only from page number indicated in  pag_from to page number indicated in pag_to in df

from pypdf import PdfReader

# Load the PDF documents
documents = []

# For each document in my dataset
for index, row in df.iterrows():

    file_path = os.path.join(DATA_PATH, row["file_name"])
    reader = PdfReader(file_path)
    doc = reader.pages[row["pag_from"]:row["pag_to"]]

    #convert each page to a Document type with metadata
    pags = []
    i = 1
    for page in doc:
        pags.append(Document(page_content=page.extract_text(), metadata={'page':(i+row["pag_from"]),'source': row["file_name"] }))
        i += 1

    documents.append(pags)

# Print the number of documents loaded
print(f"Number of documents loaded: {len(documents)}")



Number of documents loaded: 5


Check the results for violencia.pdf

In [9]:
len(documents[0])

72

In [10]:
documents[0][71]

Document(page_content="84Principios de Yogyakarta (2007). Principios sobre la aplicación de \nla legislación internacional de derechos humanos en relación con la \norientación sexual y la identidad de género. Disponible en: http://yog-\nyakartaprinciples.org/principles-sp/about/\nBlas Radi y Alejandra Sardá-Chandiramani (2016). Travesticidio / \ntransfemicidio: Coordenadas para pensar los crímenes de travestis y \nmujeres trans en Argentina. Publicación en el Boletín del Observato-\nrio de Género, https://www.aacademica.org/blas.radi/14.pdf\nRussell, Diana E.H. (2008). Femicidio: politizando el asesinato de mu-\njeres. Fortaleciendo la comprensión del femicidio: De la investigación \na la acción, Program for Appropriate Technology in Health (PATH), \nInterCambios, Medical Research Council of South Africa (MRC), and \nWorld Health Organization (WHO), http://alianzaintercambios.net/\nfiles/doc/1277249021_feminicidio-COMPLETO-01.pdf#page=43\nSegato, Rita (2016), La Guerra Contra las Mujer

## Preprocesamiento de texto

Hay palabras separadas por guion como Observato-\nrio

In [11]:
# from the list documents, delete from the text the "-\n" characters

def remove_line_breaks(documents):
  for doc_list in documents:
    for doc in doc_list:
      doc.page_content = doc.page_content.replace("-\n", "")
  return documents

documents = remove_line_breaks(documents)


In [12]:
documents[0][71]

Document(page_content="84Principios de Yogyakarta (2007). Principios sobre la aplicación de \nla legislación internacional de derechos humanos en relación con la \norientación sexual y la identidad de género. Disponible en: http://yogyakartaprinciples.org/principles-sp/about/\nBlas Radi y Alejandra Sardá-Chandiramani (2016). Travesticidio / \ntransfemicidio: Coordenadas para pensar los crímenes de travestis y \nmujeres trans en Argentina. Publicación en el Boletín del Observatorio de Género, https://www.aacademica.org/blas.radi/14.pdf\nRussell, Diana E.H. (2008). Femicidio: politizando el asesinato de mujeres. Fortaleciendo la comprensión del femicidio: De la investigación \na la acción, Program for Appropriate Technology in Health (PATH), \nInterCambios, Medical Research Council of South Africa (MRC), and \nWorld Health Organization (WHO), http://alianzaintercambios.net/\nfiles/doc/1277249021_feminicidio-COMPLETO-01.pdf#page=43\nSegato, Rita (2016), La Guerra Contra las Mujeres. Madri

## División de texto en secciones

In [13]:
# tabken from https://raw.githubusercontent.com/pixegami/rag-tutorial-v2/main/populate_database.py
def split_documents(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=80,
        length_function=len,
        is_separator_regex=False,
    )
    return text_splitter.split_documents(documents)


In [14]:
chunks = []

for doc_list in documents:
  chunks.append(split_documents(doc_list))


In [15]:
chunks

[[Document(page_content='13\nViolencias por motivos de género¿Qué son las violencias por motivos de \ngénero? \nLas violencias por motivos de género son una manifestación de las relaciones de poder históricamente desiguales entre los géneros. Esta perspectiva las enmarca como un problema social, cultural y político. Es decir, \ncuando hablamos de violencias por motivos de género no nos estamos refiriendo a un fenómeno individual, de pareja o intrafamiliar, sino a la subordinación social de mujeres, lesbianas, gays, bisexuales, trans, travestis, intersex, no binaries e identidades no heteronormadas (LGBTI+) en el marco \nde un orden de género y una política sexual que adquiere características \nparticulares en el actual escenario histórico.', metadata={'page': 14, 'source': 'violencia.pdf'}),
  Document(page_content='particulares en el actual escenario histórico. \nDe esta manera, las violencias por motivos de género no se limitan a la violencia ejercida por algunos varones hacia determ

In [16]:
# unlist chunks
chunks  = [val for sublist in chunks for val in sublist]


In [17]:
type(chunks[0])

langchain_core.documents.base.Document

In [18]:
chunks[0].metadata

{'page': 14, 'source': 'violencia.pdf'}

In [19]:
# taken from https://raw.githubusercontent.com/pixegami/rag-tutorial-v2/main/populate_database.py
def calculate_chunk_ids(chunks):

    # This will create IDs like "data/violencia.pdf:16:2"
    # Page Source : Page Number : Chunk Index

    last_page_id = None
    current_chunk_index = 0

    for chunk in chunks:
        source = chunk.metadata.get("source")
        page = chunk.metadata.get("page")
        current_page_id = f"{source}:{page}"

        # If the page ID is the same as the last one, increment the index.
        if current_page_id == last_page_id:
            current_chunk_index += 1
        else:
            current_chunk_index = 0

        # Calculate the chunk ID.
        chunk_id = f"{current_page_id}:{current_chunk_index}"
        last_page_id = current_page_id

        # Add it to the page meta-data.
        chunk.metadata["id"] = chunk_id

    return chunks

## Creación y configuración de base de datos

### Limpiar la base de datos

In [20]:
def clear_database():
    if os.path.exists(CHROMA_PATH):
        shutil.rmtree(CHROMA_PATH)


In [21]:
clear_database()

### get_embedding_function()

In [22]:
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
# create the open-source embedding function
def get_embedding_function():
    embeddings = SentenceTransformerEmbeddings(model_name=EMBED_MODEL)
    return embeddings

### Creación de base de datos usando ChromaDB

In [24]:
# Load the existing database.
db = Chroma( persist_directory=CHROMA_PATH, embedding_function=get_embedding_function())

#os.chmod(CHROMA_PATH, 777)  # Otorga permisos de lectura y escritura


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [25]:
!ls

chroma	data  sample_data


In [26]:
# Calculate Page IDs.
chunks_with_ids = calculate_chunk_ids(chunks)

In [27]:
len(chunks_with_ids)

664

In [28]:
len(chunks)

664

In [31]:
chunks_with_ids[0]

Document(page_content='13\nViolencias por motivos de género¿Qué son las violencias por motivos de \ngénero? \nLas violencias por motivos de género son una manifestación de las relaciones de poder históricamente desiguales entre los géneros. Esta perspectiva las enmarca como un problema social, cultural y político. Es decir, \ncuando hablamos de violencias por motivos de género no nos estamos refiriendo a un fenómeno individual, de pareja o intrafamiliar, sino a la subordinación social de mujeres, lesbianas, gays, bisexuales, trans, travestis, intersex, no binaries e identidades no heteronormadas (LGBTI+) en el marco \nde un orden de género y una política sexual que adquiere características \nparticulares en el actual escenario histórico.', metadata={'page': 14, 'source': 'violencia.pdf', 'id': 'violencia.pdf:14:0'})

In [33]:
ids = [chunk.metadata["id"] for chunk in chunks_with_ids]

In [34]:
ids

['violencia.pdf:14:0',
 'violencia.pdf:14:1',
 'violencia.pdf:14:2',
 'violencia.pdf:15:0',
 'violencia.pdf:15:1',
 'violencia.pdf:16:0',
 'violencia.pdf:17:0',
 'violencia.pdf:17:1',
 'violencia.pdf:18:0',
 'violencia.pdf:18:1',
 'violencia.pdf:19:0',
 'violencia.pdf:19:1',
 'violencia.pdf:19:2',
 'violencia.pdf:19:3',
 'violencia.pdf:20:0',
 'violencia.pdf:20:1',
 'violencia.pdf:20:2',
 'violencia.pdf:21:0',
 'violencia.pdf:21:1',
 'violencia.pdf:21:2',
 'violencia.pdf:21:3',
 'violencia.pdf:22:0',
 'violencia.pdf:22:1',
 'violencia.pdf:23:0',
 'violencia.pdf:23:1',
 'violencia.pdf:23:2',
 'violencia.pdf:24:0',
 'violencia.pdf:24:1',
 'violencia.pdf:24:2',
 'violencia.pdf:25:0',
 'violencia.pdf:25:1',
 'violencia.pdf:26:0',
 'violencia.pdf:26:1',
 'violencia.pdf:26:2',
 'violencia.pdf:27:0',
 'violencia.pdf:27:1',
 'violencia.pdf:27:2',
 'violencia.pdf:28:0',
 'violencia.pdf:28:1',
 'violencia.pdf:28:2',
 'violencia.pdf:29:0',
 'violencia.pdf:29:1',
 'violencia.pdf:29:2',
 'violencia

In [35]:
db.add_documents(chunks, ids=ids)
db.persist()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [38]:
# query it
query = "Tipos de violencia de género"
docs = db.similarity_search(query)

In [39]:
len(docs)

4

In [40]:
docs

[Document(page_content='79\nViolencias por motivos de género', metadata={'id': 'violencia.pdf:80:0', 'page': 80, 'source': 'violencia.pdf'}),
 Document(page_content='diversidades que permite abordar las violencias dirigidas al conjunto \nde las identidades de género.', metadata={'id': 'violencia.pdf:20:2', 'page': 20, 'source': 'violencia.pdf'}),
 Document(page_content='38Mitos sobre las violencias por motivos \nde género\nLos mitos sobre las violencias de género son creencias populares es tereotipadas que se sostienen a lo largo del tiempo. La complejidad \nde la vigencia y propagación de los mitos o falsas creencias sobre \nlas violencias de género es que las naturalizan y las relacionan con \nlo inevitable, lo instintivo, desdibujando su carácter estructural y la \nposibilidad de modificarlos. \nAl mismo tiempo, las violencias de género suelen ser ubicadas como una \nproblemática de una determinada clase social, justificadas en la falta de \neducación o en casos aislados, lo que evi

In [42]:
# query it
query = "Trabajo doméstico"
docs = db.similarity_search(query)
docs

[Document(page_content='ámbito de lo doméstico por ser sentidas como “de mujeres”, improductivas económicamente, o que no corresponde realizarlas \n“porque estuve todo el día trabajando”.', metadata={'id': 'masculinidades.pdf:21:2', 'page': 21, 'source': 'masculinidades.pdf'}),
 Document(page_content='29\nIgualdad en los cuidadosmenos recursos económicos hacen, en promedio, más trabajo no remunerado que las mujeres de mayores recursos. \nLas mujeres pobres tienen más demandas de cuidados y menos posibilidades de resolverlas si el Estado no garantiza la educación pública, \nel acceso a la salud, oportunidades laborales formales, entre otros. \nMientras que las mujeres de ingresos medios y altos pueden pagar \npor servicios de atención externa o por unx trabajadorx de casa particular y usar ese tiempo para generar otros ingresos, las mujeres pobres tienen menos opciones públicas para hacer lo mismo, limitando \nasí sus oportunidades económicas (Rodríguez E. y Marzonetto 2016).', metadata

In [46]:
PROMPT_TEMPLATE = """
Responda a la pregunta basandose únicamente en el siguiente contexto:

{context}

---

Responda la pregunta basándose en el contexto brindado antes: {question}
"""

### Ollama config & install

In [45]:
#Install package and load the extension
!pip install colab-xterm
%load_ext colabxterm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.6/115.6 kB 1.4 MB/s eta 0:00:00


Correr los siguientes comandos en la xterm

```
curl -fsSL https://ollama.com/install.sh | sh
```



```
ollama serve & ollama pull llama3
```






In [47]:
%xterm

Launching Xterm...

<IPython.core.display.Javascript object>

In [48]:

from langchain_community.llms import Ollama
llm = Ollama(model = "llama3")
llm.invoke("Tell me 3 red flower names.")

'Here are three red flower names:\n\n1. Poppy\n2. Hollyhock\n3. Tulipa (some varieties of tulips have a deep red color)'

In [49]:
query_text = "¿Es violencia de género que mi novio me revise el celular? Justifica"

In [50]:
from langchain.prompts import ChatPromptTemplate
from langchain_community.llms.ollama import Ollama

# Search the DB.
results = db.similarity_search_with_score(query_text, k=10)


In [51]:
context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, question=query_text)
print(prompt)


Human: 
Responda a la pregunta basandose únicamente en el siguiente contexto:

38Mitos sobre las violencias por motivos 
de género
Los mitos sobre las violencias de género son creencias populares es tereotipadas que se sostienen a lo largo del tiempo. La complejidad 
de la vigencia y propagación de los mitos o falsas creencias sobre 
las violencias de género es que las naturalizan y las relacionan con 
lo inevitable, lo instintivo, desdibujando su carácter estructural y la 
posibilidad de modificarlos. 
Al mismo tiempo, las violencias de género suelen ser ubicadas como una 
problemática de una determinada clase social, justificadas en la falta de 
educación o en casos aislados, lo que evita comprender la problemática 
como una cuestión pública y social (González Prado - Yanes, 2013).

---

79
Violencias por motivos de género

---

violencia política-pública.
Los distintos tipos y modalidades de violencias se sostienen en 
creencias y estereotipos vigentes que son reproducidos a través 

In [52]:
model = Ollama(model="llama3")
response_text = model.invoke(prompt)


Cambiar llama3 a tinyllama para ver si corre más rápido

In [53]:

sources = [doc.metadata.get("id", None) for doc, _score in results]
formatted_response = f"Respuesta: {response_text}\nFuentes: {sources}"
print(formatted_response)

Respuesta: En este contexto, no se considera violencia de género que tu novio te revise el celular. La violencia por motivos de género se refiere a una serie de acciones o comportamientos que buscan controlar, dominar o agredir a alguien con base en su género, identidad de género o expresión de género.

Revisar el celular sin consentimiento no es necesariamente un acto de violencia de género. Sin embargo, si este comportamiento forma parte de una patrón de control o dominación en tu relación, podría ser un indicador de problemas más graves que merecen ser abordados.

Es importante recordar que la violencia de género es un fenómeno complejo que puede involucrar a cualquier persona, independientemente de su género, identidad de género o expresión de género. La perspectiva de género y diversidad nos permite ver la problemática como una cuestión pública y social, más allá de las creencias populares o estereotipos que naturalizan la violencia.

En el contexto brindado, se destaca la importa